# Projeto de Redes Sociais

Cicero Tiago Carneiro Valentim

Lais Nascimento da Silva

Thalia Loiola da Silva

William Augusto Reis da Silva

### Bibliotecas

In [1]:
from graph_tool import centrality, clustering, PropertyMap
from graph_tool import Graph
# from graph_tool.all import graph_draw
from netpixi.data.gt import *
from netpixi import render, peek
import netpixi
import pandas as pd
import netpixi
import numpy as np

Às vezes é necessário instalar o openpyxl. Só descomentar:

In [2]:
# !pip install openpyxl

Lendo o dataset e tirando quando tem valores nulos.

Recorte por anos.

In [ ]:
df = pd.read_excel('netflix.xlsx')
df.head(2)
df = df.dropna(subset=['Genre'])

initial_date = np.datetime64('2000-01-01')
final_date = np.datetime64('2010-01-01')

# df = df[(df['Release Date'] > initial_date) & (df['Release Date'] < final_date)]
# df = df.reset_index()
df

In [ ]:
LISTA_TITLES = []
lista_number = []
titles_ind = {}
for i in range(len(df)):
    if df.loc[i]["Title"] not in LISTA_TITLES:
        LISTA_TITLES.append(df.loc[i]["Title"])
        titles_ind[df.loc[i]["Title"]] = 1
        lista_number.append(1)
        
    else:
        titles_ind[df.loc[i]["Title"]] += 1
        lista_number.append(titles_ind.get(df.loc[i]["Title"]))

In [ ]:
len(LISTA_TITLES)

In [ ]:
df["qtde"] = lista_number

In [ ]:
df = df[df["qtde"] == 1]
df = df.reset_index()

In [ ]:
# df = pd.DataFrame(df)
for linha in range(df.shape[0]):
    df['Genre'].loc[linha] = [genre.strip() for genre in df['Genre'].loc[linha].split(",")] 

In [ ]:
titles = df["Title"]
titles

In [ ]:
df

In [ ]:
df = df.explode('Genre')
df = df.reset_index(drop=True)

In [ ]:
df

In [ ]:
# pega todos os generos existentes do dataset 
genres = set()
for genre in df.Genre:
    genres.add(genre)

# transforma em lista 
genres = list(genres)

In [ ]:
df.head(3)

## Vértices

In [ ]:
g = Graph(directed=False)
g.vp['name'] = g.new_vp('object')
g.vp['bipartite'] = g.new_vp('object')

for i in range(len(titles)):
    g.add_vertex()
    g.vp['name'][i] = str(titles[i])
    g.vp['bipartite'][i] = 0

print('número de vértices de g:', g.num_vertices())

In [ ]:
len(genres) + len(titles)

In [ ]:
for i in range(len(genres)):
    g.add_vertex()
    g.vp['name'][i + len(titles)] = str(genres[i])
    g.vp['bipartite'][i + len(titles)] = 1
    
print('número de vértices de g total:', g.num_vertices())

In [ ]:
for v in g.vertices():
    print(g.vp['name'][v])

In [ ]:
lista = []
for linha in range(df.shape[0]):
    title = df['Title'].loc[linha]
    genre = df['Genre'].loc[linha]
    index_title = 0
    index_genre = 0
    v = 0
    while v < g.num_vertices() - len(genres):
        if str(g.vp['name'][v]) == str(title):
            index_title = v
            break
        v += 1
    u = len(titles)
    while u < g.num_vertices():
        if str(g.vp['name'][u]) == str(genre):
            index_genre = u
            break
        u += 1
    print(f"{title}: {index_title}, {genre}: {index_genre}")
    together = []
    together.append(index_title)
    together.append(index_genre)
    if together not in lista:
        lista.append(together)

In [ ]:
len(lista)
for i in lista:
    print(i)
    g.add_edge(i[0], i[1])
#     g.add_edge(i[0], i[1], add_missing=False)

In [ ]:
for u in g.vertices():
    print(f'vizinhos de {g.vp["name"][u]}:', end=' ')
    for v in u.all_neighbors():
        print(g.vp["name"][v], end=' ')
    print()

In [ ]:
save_gt(g, 'projeto.net.gz')

In [ ]:
g.vp['color'] = g.new_vp('object')

for v in g.vertices():
    if g.vp['bipartite'][v] == 0:
        g.vp['color'][v] = 0xff0000
    else:
        g.vp['color'][v] = 0x00ff00
top = []

for v in g.vertices():
    if g.vp['bipartite'][v] == 0:
        top.append(v)

m = draw_gt.bipartite_layout(g, top)
move_gt(g, m)
save_gt(g, 'projeto.net.gz')
# r = netpixi.render('projeto.net.gz')

In [ ]:
for i in g.vp['name']:
    if type(i) != str:
        print(type(i))
        print(g.vp["name"][i])

One Mode

In [ ]:
import numpy as np
# Criando o Grafo
grafo_titulo = Graph(directed = False)
grafo_titulo.vp['nome'] = grafo_titulo.new_vp('object')

i = 0
for v in g.vertices():
    if g.vp['bipartite'][v] == 0: # Adicionando os vértices (títulos)
        grafo_titulo.add_vertex()
        grafo_titulo.vp['nome'][i] = g.vp['name'][v]
        i+=1
    
m = np.zeros((grafo_titulo.num_vertices(), grafo_titulo.num_vertices()))
grafo_titulo.ep['weight'] = grafo_titulo.new_ep('object')
# Criando os vértices e preenchendo seu objeto de acordo com os vizinhos do grafo original
for u in g.vertices():
    
    if g.vp['bipartite'][u] == 1:
        viz = []
        for v in u.all_neighbors():
            for n in range(grafo_titulo.num_vertices()):
                if grafo_titulo.vp['nome'][n] == g.vp["name"][v]:
                    viz.append(n)
        
        print(viz)
        while len(viz) > 1:
            j = 1
            while j<len(viz):
                if m[viz[0], viz[j]] == 0:     
                    grafo_titulo.add_edge(viz[0], viz[j])
                    grafo_titulo.ep["weight"][grafo_titulo.edge(viz[0], viz[j])] = 1
                    m[viz[0], viz[j]] = 1

                else:
                    grafo_titulo.ep["weight"][grafo_titulo.edge(viz[0], viz[j])] += 1
                    m[viz[0], viz[j]] += 1
                
                j += 1
            del viz[0]

In [ ]:
# Remove da rede todas as arestas em que weight < lower.
def dicotomize(g, lower):
    removed = []

    for e in g.edges():
        if g.ep['weight'][e] < lower:
            removed.append(e)

    for e in removed:
        g.remove_edge(e)

In [ ]:
dicotomize(grafo_titulo, 4)
save_gt(grafo_titulo, 'gr_titulo.net.gz')

In [ ]:
grafo_titulo = load_gt('gr_titulo.net.gz')
m = draw_gt.sfdp_layout(grafo_titulo)
move_gt(grafo_titulo, m)
save_gt(grafo_titulo, 'exemplo2.net.gz')
# r = netpixi.render('exemplo2.net.gz', infinite=True);

In [ ]:
from graph_tool import centrality, clustering

# calculo clustering coefficient
c = clustering.local_clustering(grafo_titulo)

# calculo betweenness
bc, _ = centrality.betweenness(grafo_titulo)

grafo_titulo.vp['betweenness'] = bc
grafo_titulo.vp['clustering'] = c

In [ ]:
df = pd.DataFrame({k: list(v) for k, v in grafo_titulo.vp.items()})

In [ ]:
filmes = pd.read_excel('netflix.xlsx')

In [ ]:
df.sort_values(by='betweenness')

In [ ]:
filmes.columns

In [ ]:
df

In [ ]:
filmes_score = filmes[['IMDb Score', 'Title']].rename(columns={"Title":"nome"})

In [ ]:
filmes_score.nome = filmes_score.nome.astype(str, copy=False)
df.nome = df.nome.astype(str, copy=False)
df.id = df.id.astype(int, copy=False)

Remoção de filmes duplicados

In [ ]:
filmes_score = filmes_score.drop_duplicates(subset=['nome'], keep=False)

Adição da coluna `IMDb Score` ao datafram `df`

In [ ]:
df = df.set_index('nome').join(filmes_score.set_index('nome'), how='left').reset_index()

In [ ]:
df.nome.describe()

In [ ]:
df.head()

In [ ]:
df = df.rename(columns={"IMDb Score": 'desempenho'})

In [ ]:
df.head()

In [ ]:
import seaborn as sns
sns.set()

In [ ]:
import statsmodels.formula.api as smf

### Agrupamento

In [ ]:
df

In [ ]:
a = df[df['betweenness'] > 0]
a

In [ ]:
a.shape[0]/df.shape[0]

In [ ]:
sns.regplot(x=a['desempenho'], y=np.log(a['betweenness']))

In [ ]:
sns.regplot(x=df['betweenness'], y=np.log(df['desempenho']))

In [ ]:
model = smf.ols(data=a, formula='desempenho ~ np.log(betweenness)')
result = model.fit()
result.summary()

Como esperado após a análise dos gráficos, o R² é bem pequeno quando queremos explicar o desempenho levando em conta o __betweenness__. Além disso, podemos ver que o coeficiente é positivo, indicando que existe uma relação positiva, no entanto, o _p\_valor_ é muito grande, ou seja, não podemos rejeitar a hipótese nula de que não há relação.

### Diversidade

In [ ]:
sns.regplot(x=df['clustering'], y=df['desempenho'])

In [ ]:
model = smf.ols(data=df, formula='desempenho ~ clustering')
result = model.fit()
result.summary()

O R² é desprezível quando queremos explicar o __desempenho__ levando em conta o __clustering__. Além disso, podemos ver que o coeficiente é negativo, indicando que existe uma relação negativa, no entanto, o _p\_valor_ é muito grande, ou seja, também não podemos rejeitar a hipótese nula de que não há relação.

Após analisarmos individualmente a contribuição de cada variável, decidimos testar o impacto da combinação das duas no desempenho do filme.

In [ ]:
model = smf.ols(data=df, formula='desempenho ~ betweenness + clustering')
result = model.fit()
result.summary()

Pelo que se pode observar, a contribuição da combinação das variáveis não parece ser suficiente para explicar o desempenho de um filme. A combinação dos coeficientes com seus respectivos _p\_valores_ continua ruim, no entanto, podemos ver uma melhoria ínfima no R².

Portanto, dado que a hipótese era que, de alguma forma, o __agrupamento__ e a __diversidade__ explicavam o __desempenho__, pode-se dizer que a hipótese não se comprova estatisticamente ou que precisaríamos de um refinamento melhor no modelo.